possible commands in the terminal 
cd /raven/ptmp/rharel/thermal/raw_videos
module load ffmpeg/4.4
'ffmpeg  -i temp2/%06d.jpg -c:v libx264 -qp 0 -preset veryslow ' + vid_name + '.mp4'
'ffmpeg  -i temp2/%06d.jpg -c:v libx264 -pix_fmt yuv420p -qp 0 -preset veryslow ' + vid_name + '.mp4'

print('ffmpeg -i temp2/%06d.tiff -c:v libx264 -qp 0 -preset veryslow ' + vid_name + '.mp4')


to sort out
tiff or jpg
final ffmpeg setting
it is possible to pararalize frames preperation
save time stamps
check subsampling effect

In [ ]:
# make sure you are on Python 3.9
# possible installations
#!pip install guppy3
#!conda install -c conda-forge guppy3
#!pip uninstall FileSDK
#!pip install opencv-python
#!conda install -c conda-forge opencv
#!module load ffmpeg/4.4
#!conda install -c conda-forge opencv
#!pip install opencv-python

#installing Flir File SDK if needed 
#os.chdir('/raven/ptmp/rharel/thermal/raw_videos/sdks/file/python/dist')
#!pip install FileSDK-4.1.0-cp36-cp36m-win_amd64.whl
#!module load anaconda/3/2021.11
#os.chdir('/raven/ptmp/rharel/thermal/raw_videos/python')
#!python setup.py install --user
#os.chdir('/raven/ptmp/rharel/thermal/raw_videos')
#!python3 -m pip install --upgrade pip


In [1]:
# load packages
import os
import glob

import time
import numpy as np
import seaborn as sns
import datetime as dt
import pandas as pd
from tkinter import filedialog
from tkinter import *

import matplotlib.pyplot as plt
import matplotlib.image as plt_img
import matplotlib.image as plt_img
from matplotlib.offsetbox import AnchoredText

import h5py
import cv2
from guppy import hpy
import gc
import shlex
import pipes
from subprocess import check_call
import fnv
import fnv.reduce
import fnv.file  
from tqdm import tqdm
from joblib import Parallel, delayed

module('load','ffmpeg/4.4')
   
gc.collect()
#h5py.run_tests()

0

In [9]:
def plot_thermal_frame(i: int, it: int):
    """
    Plot a thermal frame with a timestamp and save it as a TIFF file.

    Args:
        i (int): Index of the frame.
        it (Iterator): An iterator object.

    Returns:
        None
    """
    fig = plt.figure(figsize=(width, height), dpi=100)
    
    # Get the current frame and reshape it to the height and width of the image.
    im.get_frame(it)
    final = np.array(im.final, copy=False).reshape((im.height, im.width))
    final1 = final - final.mean()
    
    # Plot the thermal image with rocket colormap.
#     plt.imshow(final1, vmin=-np.std(final1), vmax=final1.max(), cmap='rocket')
    plt.imshow(final1,vmin = -np.quantile(final1,.5), vmax = final1.max(), cmap='rocket',interpolation='catrom')
    
    # Add timestamp to the plot.
    timestamp = im.frame_info[0]['value'][4:]
    timestamp = dt.datetime.strptime(timestamp, '%H:%M:%S.%f')
    text_box = AnchoredText(timestamp.strftime("%H:%M:%S.%f")[:-5] + "_" + str(i), frameon=True, loc=2, pad=0.5,
                            prop=dict(fontsize=12))
    plt.setp(text_box.patch, facecolor='white', alpha=0.2)
    plt.gca().add_artist(text_box)
    
    # Set axis and save the plot as a TIFF file.
    plt.gca().set_position([0, 0, 1, 1])
    plt.axis('off')
    plt.savefig(f'{i:06}' +  '.tiff', bbox_inches='tight', pad_inches=0)
    
    plt.close()

In [3]:
## writing all frames from a given thermal recording, separated into minute bins. Timestamps unrounded
zoom = 1
vid_length = 1*60*60

main_folder = '/raven/ptmp/rharel/thermal/raw_videos/'
image_folder = main_folder + 'temp/'
data_folder =  main_folder + 'viewpoint_1/T1020/20190818/'
output_folder =  main_folder + 'output/'

if not os.path.isdir( image_folder ):
    os.makedirs ( image_folder)

if not os.path.isdir( output_folder ):
    os.makedirs ( output_folder)
    
os.chdir(data_folder)
file_names = glob.glob('*.seq')
file_names

['20190807_17_00_00-219_14_00_05_282.seq',
 '20190813_16_30_00-225_13_30_05_916.seq',
 '20190812_16_56_00-224_13_56_02_574.seq',
 '20190803_16_30_00-215_13_30_06_187.seq',
 '20190816_16_30_00-228_13_30_06_090.seq',
 '20190801_17_06_00-213_14_06_02_088.seq',
 '20190809_16_30_00-221_13_30_08_080.seq',
 '20190817_17_00_00-229_14_00_02_320.seq',
 '20190802_16_05_00-000008-214_13_05_07_052.seq',
 '20190810_16_30_00-222_13_30_02_972.seq',
 '20190818_17_11_00-__TIME__.seq',
 '20190804_16_30_00-216_13_30_09_112.seq',
 '20190811_16_30_00-223_13_30_05_954.seq',
 '20190731_18_00_00-212_15_00_08_934.seq',
 '20190814_17_00_00-226_14_00_04_981.seq',
 '20190821_16_01_00-233_13_01_00_924.seq',
 '20190808_16_30_00-220_13_30_05_390.seq',
 '20190819_16_20_00-231_13_20_01_056.seq',
 '20190815_16_35_00-227_13_35_05_016.seq',
 '20190820_16_23_00-232_13_23_02_966.seq']

In [25]:
image_folder = main_folder + vid_name + '/'
image_folder


'/raven/ptmp/rharel/thermal/raw_videos/20190818_30253_5/'

In [ ]:
for file_name in file_names:
    im = fnv.file.ImagerFile(data_folder + file_name)

    im.get_frame(0)
    #im.frame_info[1]
    #specifying first and last frame after loading the specific file
    print('got_frame')

    # selected frames to make the video
    start_frame = 0
    end_frame = im.num_frames
    step_frame = 5
    frame_range = range(start_frame,end_frame,step_frame)
    print(frame_range)

    vid_name = file_name[:8] + '_' + str(end_frame) + '_' + str(step_frame)
    if not os.path.exists(output_folder + vid_name + '.mp4'):
        # where to write the images to   
        #day1 = int(im.frame_info[0]['value'][:3])
        #start = f'{start_dt.year:04}-' + f'{start_dt.month:02}-' + f'{start_dt.day:02} ' + im.frame_info[0]['value'][4:]
        #start_dt = dt.datetime.strptime(start, '%Y-%m-%d %H:%M:%S.%f')

        os.chdir( image_folder )
        for f in glob.glob(image_folder + "*.tiff"):
            os.remove(f)

        final = np.array(im.final, copy=False).reshape((im.height, im.width))
        width = final.shape[1] / (100 / zoom)  
        height = final.shape[0] / (100 / zoom)

        fig = plt.figure( figsize = ( width, height ) )  
        fig.set_dpi(100)

#         if start_frame > 0:
#             counter = 0
#         else:
#             counter = 1

#         print((im.width, im.height))


        # Loop through frame_range and make frames with plot_thermal_frame function
        for idx, item in tqdm(enumerate(frame_range)):
            plot_thermal_frame(idx, item)

        del im
        gc.collect()

        os.chdir(main_folder)
        command = 'ffmpeg -i ' + image_folder + '%06d.tiff -c:v libx264 -qp 0 '  + output_folder + vid_name + '.mp4'
        check_call(shlex.split(command))
        

In [ ]:
file_name = file_names[0]
#im = fnv.file.ImagerFile(data_folder + file_names[0])

#im.get_frame(0)
# im.frame_info[1]
#specifying first and last frame after loading the specific file
#print('got_frame')

# selected frames to make the video
start_frame = 0
end_frame = im.num_frames
step_frame = 25
frame_range = range(start_frame,end_frame,step_frame)
print(frame_range)

vid_name = file_name[:8] + '_' + str(end_frame) + '_' + str(step_frame)


In [ ]:

im.get_frame(120000)

zoom = .5
dpi = 200
np.shape(im.final)
#data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
data = np.array(im.final)
data = np.reshape(data,(im.height, im.width)) 
data = data - data.mean()

width = data.shape[1] / (100 / zoom)  
height = data.shape[0] / (100 / zoom)

# data = data[300:400,100:200]  

methods = [None, 'none', 'nearest', 'bilinear', 'bicubic', 'spline16',
           'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric',
           'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos']

fig = plt.figure( figsize = ( width, height), dpi = dpi )  
plt.imshow(data,vmin = -np.quantile(data,.5), vmax = data.max(), cmap='rocket',interpolation='catrom')
#plt.imshow(data,vmin = -np.std(data), vmax = data.max(), cmap='rocket',interpolation='catrom')

plt.show()

In [ ]:
# example frames for testing 

i = 40
fig = plt.figure( figsize = ( width, height), dpi = 300 )  
fig.set_dpi(100)
#im.get_frame(i)
im.get_frame(i)
#final = np.array(im.final, copy=False).reshape( ( im.height, im.width ) )
#final1 = final - final.mean()
plt.imshow(final1, vmin = -np.std(final1), vmax = final1.max(), cmap='rocket') # rocket # interpolation='nearest' 
#sns.heatmap(final1[i], vmin = -np.std(final1[i]), vmax = final1[i].max(), xticklabels = False, yticklabels = False, cbar = False)

timestamp = im.frame_info[0]['value'][4:]
timestamp = dt.datetime.strptime(timestamp, '%H:%M:%S.%f')
text_box = AnchoredText( timestamp.strftime("%H:%M:%S:%f")[:-5] , frameon=True, loc=2, pad=0.5,prop=dict(fontsize=12))
plt.setp(text_box.patch, facecolor='white', alpha=0.2)
plt.gca().add_artist(text_box)
plt.axis('off')
fig.canvas.draw()

#plt.savefig( f'{i:06}' +  '.tiff' , pad_inches = 0) # f'{i:06}_' + timestamps[i].strftime("%H%M%S%f")[:-3]
data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
data1 = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))


im.get_frame(120000)

zoom = .5
dpi = 300
np.shape(im.final)
#data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
data = np.array(im.final)
data = np.reshape(data,(im.height, im.width)) 
data = data - data.mean()

width = data.shape[1] / (100 / zoom)  
height = data.shape[0] / (100 / zoom)

data = data[500:600,500:600]  

fig = plt.figure( figsize = ( width, height), dpi = dpi )  
plt.imshow(data,vmin = -np.std(data), vmax = data.max(), cmap='rocket')
plt.show()